In [1]:
import pandas as pd

tagwiki = pd.read_csv("C:/Users/arisa/StackOverflow-Project/dataset/tagwiki.csv",skiprows = 1)
tagwiki

,excerpt_last_edit_date,body_last_edit_date,excerpt,tag_name
0,1371244527,1450439435,A static library of object code in UNIX/Linux ...,.a
1,1353649971,1362982922,.app is an extension given to application bund...,.app
2,1342836577,1495770630,"A (typically short) shell script, executed whe...",.bash-profile
3,1339834647,1339828227,Class files are the format used by the Java Vi...,.class-file
4,1300892169,1528129773,A file extension for word processing documents...,.doc
...,...,...,...,...
41612,1473614421,1473614421,Use this tag for questions relating to Joyride...,zurb-joyride
41613,1323089468,1394720490,ZXing (pronounced &quot;zebra crossing&quot;) ...,zxing
41614,1512552477,1512578291,zxing.net is a library which supports decoding...,zxing.net
41615,1419378735,1419378378,Zynq refers to the Zynq-7000 family of SoCs. A...,zynq


In [2]:
import string
import re
import pandas as pd
import nltk 

def extract_nn(excerpt, grammar, max_num = 2):
    grammar = "NP: {<NN|NNS>+}"
    cp = nltk.RegexpParser(grammar)
    sentences = nltk.sent_tokenize(excerpt)
    first_sent_tokens = nltk.word_tokenize(sentences[0])
    first_sent_tokens = [token for token in first_sent_tokens if token not in string.punctuation]
    first_sent_tags = nltk.pos_tag(first_sent_tokens)
    result = cp.parse(first_sent_tags)
    all_tags = []
    count = 0
    for node in result:
        try:
            label = node.label()
            if (label == "NP") and (count < max_num):
                leaf_nodes = [x[0] for x in node.leaves() if x[1] in ["NNS", "NN"]]
                all_tags += leaf_nodes
                count += 1
        except AttributeError:
            continue
    return all_tags

for i in range(0,20807):
    sentences = tagwiki['excerpt']
    #print(sentences[i])
    nan = tagwiki[tagwiki['excerpt'].isnull()].index.tolist()
    if (i in nan) or (len(tagwiki['excerpt'][i].strip()) == 0):
        continue
    categories = extract_nn(sentences[i], grammar="NP: {<NN|NNS>+}")
    

In [4]:
for i in range(0,20807):
    sentences = tagwiki['excerpt']
    #print(sentences[i])
    nan = tagwiki[tagwiki['excerpt'].isnull()].index.tolist()
    if (i in nan) or (len(tagwiki['excerpt'][i].strip()) == 0):
        continue
    categories = extract_nn(sentences[i], grammar="NP: {<NN|NNS>+}")
    tagdict = {tagwiki['tag_name'][i]:categories}

In [5]:
nan = tagwiki[tagwiki['excerpt'].isnull()].index.tolist()
tag_dict = {}
for i in range(0,20807):
    sentences = tagwiki['excerpt']
    if (i in nan) or (len(tagwiki['excerpt'][i].strip()) == 0):
        continue
    categories = extract_nn(sentences[i], grammar="NP: {<NN|NNS>+}")
    #tagdict = dict({tagwiki['tag_name'][i]:categories})
        
    key = tagwiki['tag_name'][i]
    val = categories
    tag_dict.update({key:val}) 


In [5]:
len(tag_dict)

19404

In [6]:
tag_dict.values()

dict_values([['library', 'code'], ['.app', 'extension'], ['shell', 'script', 'bash', 'user', 'logs'], ['Class', 'files', 'format'], ['file', 'extension', 'word', 'processing', 'documents'], ['configuration', 'file', 'web', 'servers'], ['.htpasswd', 'usernames'], ['file', 'format', 'image', 'file', 'format'], ['.lib', 'extension'], ['.mov', 'file', 'extension'], ['version'], ['version'], ['version', 'Framework'], [], ['version', 'framework'], [], ['framework'], ['version', 'framework'], ['.NET', 'framework', 'release'], [], ['group', 'classes'], ['framework', 'version', 'questions'], ['.NET', 'cross-platform', 'development', 'platform'], ['tag', 'questions'], ['use', 'version'], ['.NET', 'successor'], ['questions', 'versions'], ['toolkit', 'devices'], ['set', 'libraries'], ['tag', 'questions'], ['.NET', 'legacy', 'technology'], ['version'], ['specification', 'runtimes'], ['questions', 'use', '.net-core'], ['file', 'contains', 'data', 'host'], ['file', 'format', 'stores'], ['jQuery', 'wr

In [6]:
category_dict = {}
nan = tagwiki[tagwiki['excerpt'].isnull()].index.tolist()
for i in range(0,20807):
    sentences = tagwiki["excerpt"][i]
    tag_name = tagwiki["tag_name"][i]
    if (i in nan) or (len(tagwiki["excerpt"][i].strip()) == 0):
        continue
    categories = extract_nn(sentences, grammar="NP: {<NN|NNS>+}")
    for category in categories:
        if category in category_dict:
            category_dict[category].append(tag_name)
        else:
            category_dict[category] = [tag_name]

In [8]:
import json

with open('tag_dict.json', 'w') as fp:
    json.dump(tag_dict, fp,  indent=4)

In [7]:
import json

with open('category_dict.json', 'w') as fp:
    json.dump(category_dict, fp, sort_keys=True, indent=4)